# TP de Graphe :
# 1.1 Construction du graphe

In [2]:
import matplotlib.pyplot as plt

class Graphe:
    def __init__(self):
        self.graphe = {}
        self.depart = None
        self.arrivee = None

    def ajouter_sommet(self, sommet):
        if sommet not in self.graphe:
            self.graphe[sommet] = set()

    def ajouter_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].add(v)
            self.graphe[v].add(u)
    
    def ajouter_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].add(v)
            self.graphe[v].add(u)

    def supprimer_arete(self, u, v):
        if u in self.graphe and v in self.graphe:
            self.graphe[u].discard(v)
            self.graphe[v].discard(u)

    def creer_graphe_depuis_fichier(self, chemin_fichier):
        with open(chemin_fichier, 'r') as fichier:
            lignes = fichier.readlines()
        
        n, m = map(int, lignes[0].split())
        matrice = [ligne.split() for ligne in lignes[1:]]
        
        for y in range(n):
            for x in range(m):
                if matrice[y][x] in {'1', '2', '3'}:
                    self.ajouter_sommet((x, y))
                    if matrice[y][x] == '2':
                        self.depart = (x, y)
                    if matrice[y][x] == '3':
                        self.arrivee = (x, y)
                
                # Ajouter les arêtes pour les sommets voisins dans le graphe
                if matrice[y][x] != '0':
                    if x > 0 and matrice[y][x-1] != '0':  # À gauche
                        self.ajouter_arete((x, y), (x - 1, y))
                    if y > 0 and matrice[y-1][x] != '0':  # En haut
                        self.ajouter_arete((x, y), (x, y - 1))
    
    def afficher_matrice(self):
        # Déterminer la taille de la matrice
        max_x = max(self.graphe, key=lambda x: x[0])[0] + 1
        max_y = max(self.graphe, key=lambda x: x[1])[1] + 1
        
        # Initialiser la matrice avec des zéros
        matrice = [['0' for _ in range(max_x)] for _ in range(max_y)]
        
        # Remplir la matrice avec les chemins (1), départ (2) et arrivée (3)
        for (x, y) in self.graphe:
            matrice[y][x] = '1'
        if self.depart:
            matrice[self.depart[1]][self.depart[0]] = '2'
        if self.arrivee:
            matrice[self.arrivee[1]][self.arrivee[0]] = '3'
        
        # Afficher la matrice
        for ligne in matrice:
            print(' '.join(ligne))

    def tracer_graphe(self):
        plt.figure(figsize=(8, 6))
        # Dessiner les arêtes
        for u in self.graphe:
            for v in self.graphe[u]:
                plt.plot([u[0], v[0]], [u[1], v[1]], 'b-', linewidth=1)  # lignes bleues pour les arêtes

        # Dessiner les sommets
        for u in self.graphe:
            plt.plot(u[0], u[1], 'ko', markersize=8)  # points noirs pour les sommets

        # Mettre en évidence les points de départ et d'arrivée
        if self.depart:
            plt.plot(self.depart[0], self.depart[1], 'go', markersize=10)  # Vert pour départ
        if self.arrivee:
            plt.plot(self.arrivee[0], self.arrivee[1], 'ro', markersize=10)  # Rouge pour arrivée

        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Visualisation du Graphe')
        plt.gca().invert_yaxis()  # Inverser l'axe Y pour correspondre au format de matrice
        plt.grid(True)
        plt.show()
    


# Exemple d'utilisation

# Usage example
graphe = Graphe()
graphe.creer_graphe_depuis_fichier('exo1.txt')  # Assurez-vous que le chemin d'accès est correct
graphe.afficher_matrice()




1 2 1 1 1 1 1 1 0
1 0 1 1 1 1 0 1 1
1 1 1 0 0 1 0 1 1
1 0 1 1 1 1 3 1 1


In [ ]:
graphe.tracer_graphe()

# 1.2 Modélisation mathématique

## Modélisation du Problème en Programmation Linéaire

Le problème du plus court chemin dans un graphe peut être formulé comme un problème de programmation linéaire. Ce problème consiste à trouver le chemin le moins coûteux entre une source et une destination dans un graphe.

### Variables
Pour chaque arête $(u, v)$ dans le graphe, nous définissons une variable binaire $x_{uv}$ qui indique si l'arête $(u, v)$ est utilisée dans le chemin le plus court.

### Fonction Objectif
La fonction objectif est de minimiser le coût total du chemin. Cela est exprimé comme suit :

$$ \min \sum_{(u, v) \in E} c_{uv} x_{uv} $$

où $c_{uv}$ représente le coût de l'arête $(u, v)$.

### Contraintes

#### Conservation du Flux
Pour chaque sommet $v$ autre que la source $s$ et la destination $t$, la conservation du flux doit être respectée :

$$ \sum_{(u, v) \in E} x_{uv} - \sum_{(v, w) \in E} x_{vw} = 0 $$

#### Contraintes de la Source et de la Destination
La source $s$ doit avoir exactement une arête sortante active et la destination $t$ doit avoir exactement une arête entrante active :

- Pour la source $s$ :
$$ \sum_{(s, v) \in E} x_{sv} = 1 $$

- Pour la destination $t$ :
$$ \sum_{(u, t) \in E} x_{ut} = 1 $$

## Résumé
Cette modélisation permet de transformer le problème du plus court chemin en un problème de programmation linéaire, où la solution optimale indique le chemin le plus court en termes de coût entre deux points donnés dans un réseau.


## Implémentation avec CPLEX

In [ ]:
from cplex import Cplex
from cplex.exceptions import CplexError

def resoudre_cplex(graphe):
    try:
        prob = Cplex()
        prob.set_problem_type(Cplex.problem_type.LP)  # Définir le problème comme linéaire

        # Ajouter les variables x_uv
        edge_vars = []
        for u in graphe.graphe:
            for v in graphe.graphe[u]:
                varname = f"x_{u}_{v}"
                prob.variables.add(names=[varname], lb=[0], ub=[1], types=["B"])
                edge_vars.append((varname, u, v))

        # Définir la fonction objectif
        prob.objective.set_sense(prob.objective.sense.minimize)
        costs = [1] * len(edge_vars)  # Mettre les coûts ici, 1 pour exemple simplifié
        varnames = [var[0] for var in edge_vars]
        prob.objective.set_linear(list(zip(varnames, costs)))

        # Ajouter les contraintes de conservation du flux
        for node in graphe.graphe:
            if node != graphe.depart and node != graphe.arrivee:
                row = [[], []]
                for varname, u, v in edge_vars:
                    if u == node:
                        row[0].append(varname)
                        row[1].append(1)
                    if v == node:
                        row[0].append(varname)
                        row[1].append(-1)
                prob.linear_constraints.add(lin_expr=[row], senses=["E"], rhs=[0])

        # Contraintes pour la source et la destination
        source_constraints = [[], []]
        sink_constraints = [[], []]
        for varname, u, v in edge_vars:
            if u == graphe.depart:
                source_constraints[0].append(varname)
                source_constraints[1].append(1)
            if v == graphe.arrivee:
                sink_constraints[0].append(varname)
                sink_constraints[1].append(1)
        prob.linear_constraints.add(lin_expr=[source_constraints], senses=["E"], rhs=[1])
        prob.linear_constraints.add(lin_expr=[sink_constraints], senses=["E"], rhs=[1])

        prob.solve()  # Résoudre le problème

        # Afficher la solution
        solution = prob.solution.get_values()
        for idx, value in enumerate(solution):
            if value > 0.9:  # Si x_uv est 1
                print(f"{edge_vars[idx]} is in the solution")

    except CplexError as exc:
        print(exc)
        return

resoudre_cplex(graphe)


## Sauvegarde de la solution